In [ ]:
import numpy as np

def listtoarray2(data): # list의 각 element의 행이 1일 때 사용. Vertical 방향으로 stack된 형태의 array 출력됨.
    data_out = []
    if len(data) != 1:
        for count in range(len(data)):
            if count == 0:
                data_out = data[count]
            else:
                data_out = np.vstack((data_out, data[count]))
    else:
        data_out = data[0]
    return(data_out)

def parsingdata(data, check_num):
    temp_raw = np.array(data)
    data_out = []
    temp = np.zeros((1,temp_raw[0,0,:].shape[0]))
    for i in range(temp_raw.shape[1]):
        if np.float64(temp_raw[0,i,0]).item() == check_num:
            temp_comp = np.delete(temp, 0, 0)
            data_out.append(temp_comp)
            temp = np.zeros((1,temp_raw[0,0,:].shape[0]))
            continue
        else:
            temp2 = np.reshape(temp_raw[0,i,:],(1,-1))
            temp = np.append(temp,temp2,axis=0)
            
    return(data_out)

def auxiliary_input(list_trainset):
    output = [] # list of last step input
    for i in range(len(list_trainset)):
        temp = list_trainset[i]
        output.append(temp[-1,:])
        
    return(output)

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import load_model
import pandas as pd

input_train = [np.array(pd.read_csv('Access_GPStraining_sensor.csv', header=None))]
output_train = [np.array(pd.read_csv('Access_GPStraining_GPS.csv', header=None))]

# parsing with certain number
check_num = 10000.0
# for training data
input_trainset = parsingdata(input_train, check_num)
aux_input_trainset = auxiliary_input(input_trainset)
aux_input_trainset = listtoarray2(aux_input_trainset) # model.fit의 입력으로 list 타입이 아닌 np.array 타입으로 넣어야 함.
output_trainset = parsingdata(output_train, check_num)
output_trainset = listtoarray2(output_trainset) # model.fit의 입력으로 list 타입이 아닌 np.array 타입으로 넣어야 함.

# Zero-padding
input_trainset = sequence.pad_sequences(input_trainset, dtype='float32', padding='pre')

In [6]:
from keras.initializers import glorot_normal
from keras.models import Model, load_model
from keras.layers import Input, Dense, Flatten, Reshape, concatenate, Multiply, Add, LSTM, Lambda
from keras.optimizers import Nadam, Adamax, Adam

import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import time
import math
import os
import csv

def GPSestimation(columns_size):
    with tf.device('/gpu:0'):
        sensor_input = Input(shape=(None,columns_size), name='main_input')
        hidden1 = LSTM(128, return_sequences=True)(sensor_input)
        hidden2 = LSTM(128)(hidden1)
        
        auxiliary_sensor_input = Input(shape=(columns_size,), name='last_step_input')
        
        r = concatenate([hidden2, auxiliary_sensor_input])
        
        y1 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(r)
        y2 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y1)
        y3 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y2)
        y4 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y3)
        y5 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y4)
        y6 = Dense(128, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y5)

        GPS_output = Dense(1, activation = 'sigmoid', kernel_initializer=glorot_normal(seed=1))(y6)

        GPS_model = Model(inputs = [sensor_input, auxiliary_sensor_input], outputs = [GPS_output])
        
    return(GPS_model)

def weightexport_LSTM(model, layer_ind):
    for i in range(len(layer_ind)):
        LSTM_weight_x = model.layers[layer_ind[i]].get_weights()[0]
        LSTM_weight_h = model.layers[layer_ind[i]].get_weights()[1]
        LSTM_bias = model.layers[layer_ind[i]].get_weights()[2]
        pd.DataFrame(LSTM_weight_x).to_csv('LSTM_weight_x'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(LSTM_weight_h).to_csv('LSTM_weight_h'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(LSTM_bias).to_csv('LSTM_bias'+str(layer_ind[i])+'.csv', header = False, index = False)
        
def weightexport_Dense(model, layer_ind):
    for i in range(len(layer_ind)):
        Dense_weight = model.layers[layer_ind[i]].get_weights()[0]
        Dense_bias = model.layers[layer_ind[i]].get_weights()[1]
        pd.DataFrame(Dense_weight).to_csv('Dense_weight'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(Dense_bias).to_csv('Dense_bias'+str(layer_ind[i])+'.csv', header = False, index = False)

In [7]:
GPS_model = GPSestimation(10) # model의 입력은 사용된 센서 개수를 의미함.
GPS_model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])
GPS_model.summary()

# model compile 후 실제로 학습되는 과정: model.fit
GPS_model.fit({'main_input': input_trainset, 'last_step_input': aux_input_trainset}, output_trainset, epochs=10, batch_size=2640) # 2640

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, None, 10)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 128)    71168       main_input[0][0]                 
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          131584      lstm[0][0]                       
__________________________________________________________________________________________________
last_step_input (InputLayer)    [(None, 10)]         0                                            
______________________________________________________________________________________________

In [3]:
print(input_trainset)

[[[ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  ...
  [-0.92816  -0.832    -0.62677  ... -1.5951   -1.3321   -1.3627  ]
  [-0.92816  -0.832    -0.6383   ... -1.5855   -1.3321   -1.3568  ]
  [-0.92816  -0.83721  -0.6383   ... -1.5855   -1.3263   -1.3568  ]]

 [[ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  ...
  [-0.93362  -0.83721  -0.6383   ... -1.5855   -1.3321   -1.3568  ]
  [-0.93908  -0.83721  -0.6383   ... -1.5903   -1.3379   -1.3568  ]
  [-0.93908  -0.84762  -0.62677  ... -1.5855   -1.3379   -1.3598  ]]

 [[ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.       ...  0.        0.        0.      ]
  [ 0.        0.        0.      

In [4]:
print(aux_input_trainset)

[[-0.92816  -0.83721  -0.6383   ... -1.5855   -1.3263   -1.3568  ]
 [-0.93908  -0.84762  -0.62677  ... -1.5855   -1.3379   -1.3598  ]
 [-0.93908  -0.83721  -0.6383   ... -1.5903   -1.3321   -1.3656  ]
 ...
 [-0.63469  -0.52536   2.5604   ...  0.086002 -0.88336  -0.94212 ]
 [-0.62238  -0.50823   2.5346   ...  0.077621 -0.87512  -0.93749 ]
 [-0.63469  -0.50823   2.5346   ...  0.077621 -0.86689  -0.93749 ]]


In [5]:
print(output_trainset)

[[0.0054776]
 [0.0057433]
 [0.0057945]
 ...
 [0.99285  ]
 [0.99485  ]
 [0.99701  ]]


In [18]:
print(type(aux_input_trainset))

<class 'numpy.ndarray'>


In [19]:
print(type(input_trainset))

<class 'numpy.ndarray'>


In [20]:
print(type(output_trainset))

<class 'numpy.ndarray'>


In [21]:
print(output_trainset[0])

['0.0054776']


In [22]:
print(type(output_trainset[0]))

<class 'numpy.ndarray'>


In [23]:
print(len(input_trainset))

26400


In [32]:
print(input_train)

[array([['-0.92269', '-0.83721', '-0.62677', ..., '-1.6047', '-1.3437',
        '-1.3802'],
       ['-0.91177', '-0.83721', '-0.62677', ..., '-1.6143', '-1.3554',
        '-1.3802'],
       ['-0.91723', '-0.84241', '-0.6383', ..., '-1.6191', '-1.3554',
        '-1.3743'],
       ...,
       ['-0.62853', '-0.50823', '2.5346', ..., '0.077621', '-0.87512',
        '-0.94212'],
       ['-0.63469', '-0.50823', '2.5346', ..., '0.077621', '-0.86689',
        '-0.93749'],
       ['10000', '10000', '10000', ..., '10000', '10000', '10000']],
      dtype='<U11')]


In [34]:
from numpy import genfromtxt
my_data = genfromtxt('Access_GPStraining_sensor.csv', delimiter=',')

In [35]:
print(my_data)

[[-9.2269e-01 -8.3721e-01 -6.2677e-01 ... -1.6047e+00 -1.3437e+00
  -1.3802e+00]
 [-9.1177e-01 -8.3721e-01 -6.2677e-01 ... -1.6143e+00 -1.3554e+00
  -1.3802e+00]
 [-9.1723e-01 -8.4241e-01 -6.3830e-01 ... -1.6191e+00 -1.3554e+00
  -1.3743e+00]
 ...
 [-6.2853e-01 -5.0823e-01  2.5346e+00 ...  7.7621e-02 -8.7512e-01
  -9.4212e-01]
 [-6.3469e-01 -5.0823e-01  2.5346e+00 ...  7.7621e-02 -8.6689e-01
  -9.3749e-01]
 [ 1.0000e+04  1.0000e+04  1.0000e+04 ...  1.0000e+04  1.0000e+04
   1.0000e+04]]


In [36]:
my_data2 = [my_data]
print(my_data2)

[array([[-9.2269e-01, -8.3721e-01, -6.2677e-01, ..., -1.6047e+00,
        -1.3437e+00, -1.3802e+00],
       [-9.1177e-01, -8.3721e-01, -6.2677e-01, ..., -1.6143e+00,
        -1.3554e+00, -1.3802e+00],
       [-9.1723e-01, -8.4241e-01, -6.3830e-01, ..., -1.6191e+00,
        -1.3554e+00, -1.3743e+00],
       ...,
       [-6.2853e-01, -5.0823e-01,  2.5346e+00, ...,  7.7621e-02,
        -8.7512e-01, -9.4212e-01],
       [-6.3469e-01, -5.0823e-01,  2.5346e+00, ...,  7.7621e-02,
        -8.6689e-01, -9.3749e-01],
       [ 1.0000e+04,  1.0000e+04,  1.0000e+04, ...,  1.0000e+04,
         1.0000e+04,  1.0000e+04]])]


In [39]:
print(input_train)

[array([[-9.2269e-01, -8.3721e-01, -6.2677e-01, ..., -1.6047e+00,
        -1.3437e+00, -1.3802e+00],
       [-9.1177e-01, -8.3721e-01, -6.2677e-01, ..., -1.6143e+00,
        -1.3554e+00, -1.3802e+00],
       [-9.1723e-01, -8.4241e-01, -6.3830e-01, ..., -1.6191e+00,
        -1.3554e+00, -1.3743e+00],
       ...,
       [-6.2853e-01, -5.0823e-01,  2.5346e+00, ...,  7.7621e-02,
        -8.7512e-01, -9.4212e-01],
       [-6.3469e-01, -5.0823e-01,  2.5346e+00, ...,  7.7621e-02,
        -8.6689e-01, -9.3749e-01],
       [ 1.0000e+04,  1.0000e+04,  1.0000e+04, ...,  1.0000e+04,
         1.0000e+04,  1.0000e+04]])]


In [45]:
print(output_train)

[array([[5.4776e-03],
       [1.0000e+04],
       [5.7433e-03],
       ...,
       [1.0000e+04],
       [9.9701e-01],
       [1.0000e+04]])]


In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)